# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [90]:
import pandas as pd
import numpy as np

from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [184]:
prod_customer = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
prod_customer.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [276]:
cust_mat = prod_customer.pivot_table(index=['ProductName'], columns=['CustomerID'], 
                            values=['Quantity'], aggfunc='sum').fillna(0)

cust_mat.head()

Quantity                                \
CustomerID                           33    200   264   356   412   464     
ProductName                                                                
Anchovy Paste - 56 G Tube              0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mini Egg Roll, Shrimp      0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mushroom Tart              0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Sausage Rolls              0.0   0.0   0.0   0.0   0.0   0.0   
Apricots - Dried                       1.0   0.0   0.0   0.0   1.0   0.0   

                                                           ...              \
CustomerID                        477   639   649   669    ... 97697 97753   
ProductName                                                ...               
Anchovy Paste - 56 G Tube           0.0   1.0   0.0   0.0  ...   0.0  25.0   
Appetizer - Mini Egg Roll, Shrimp   0.0   0.0   0.0   0.0  ...  25.0  25.0   
Appetizer - Mushroom Tart           0.0   1.0   0.0   0.0  ...  25.0   0.0   
Appetizer - Sausage Rolls           0.0   0.0   0.0   0.0  ...   0.0   0.0   
Apricots - Dried                    0.0   0.0   0.0   0.0  ...   0.0  25.0   

                                                                             \
CustomerID                        97769 97793 97900 97928 98069 98159 98185   
ProductName                                                                   
Anchovy Paste - 56 G Tube           0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mini Egg Roll, Shrimp   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mushroom Tart           0.0   0.0   0.0   0.0   0.0   0.0  25.0   
Appetizer - Sausage Rolls           0.0   0.0  25.0  25.0  25.0   0.0  25.0   
Apricots - Dried                    0.0   0.0   0.0   0.0   0.0   0.0   0.0   

                                         
CustomerID                        98200  
ProductName                              
Anchovy Paste - 56 G Tube           0.0  
Appetizer - Mini Egg Roll, Shrimp   0.0  
Appetizer - Mushroom Tart           0.0  
Appetizer - Sausage Rolls           0.0  
Apricots - Dried                    0.0  

[5 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [191]:
distances = pd.DataFrame(1/(1 + squareform(pdist(cust_mat.T, 'euclidean'))), 
                         index=cust_mat.columns, columns=cust_mat.columns)

#distances.reset_index(drop=True, inplace=True)
distances.columns = distances.columns.droplevel()

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [192]:
distances[33].head()

          CustomerID
Quantity  33            1.000000
          200           0.077421
          264           0.087047
          356           0.081800
          412           0.080634
Name: 33, dtype: float64

In [196]:
similarities = distances[33].sort_values(ascending=False)[1:]
similarities5 = similarities.head(5)
similarities5

          CustomerID
Quantity  264           0.087047
          3535          0.087047
          3317          0.087047
          2503          0.085983
          3305          0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [264]:
prod_customer.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [227]:
top_ids=list(dict(similarities5['Quantity']).keys())
top_ids

[264, 3535, 3317, 2503, 3305]

In [233]:
top_cust_prod = prod_customer[prod_customer['CustomerID'].isin(top_ids)]
top_cust_prod.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [269]:
ranked=top_cust_prod.groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False)
ranked.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [260]:
cust_mat.head()

Quantity                                \
CustomerID                           33    200   264   356   412   464     
ProductName                                                                
Anchovy Paste - 56 G Tube              0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mini Egg Roll, Shrimp      0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mushroom Tart              0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Sausage Rolls              0.0   0.0   0.0   0.0   0.0   0.0   
Apricots - Dried                       1.0   0.0   0.0   0.0   1.0   0.0   

                                                           ...              \
CustomerID                        477   639   649   669    ... 97697 97753   
ProductName                                                ...               
Anchovy Paste - 56 G Tube           0.0   1.0   0.0   0.0  ...   0.0  25.0   
Appetizer - Mini Egg Roll, Shrimp   0.0   0.0   0.0   0.0  ...  25.0  25.0   
Appetizer - Mushroom Tart           0.0   1.0   0.0   0.0  ...  25.0   0.0   
Appetizer - Sausage Rolls           0.0   0.0   0.0   0.0  ...   0.0   0.0   
Apricots - Dried                    0.0   0.0   0.0   0.0  ...   0.0  25.0   

                                                                             \
CustomerID                        97769 97793 97900 97928 98069 98159 98185   
ProductName                                                                   
Anchovy Paste - 56 G Tube           0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mini Egg Roll, Shrimp   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mushroom Tart           0.0   0.0   0.0   0.0   0.0   0.0  25.0   
Appetizer - Sausage Rolls           0.0   0.0  25.0  25.0  25.0   0.0  25.0   
Apricots - Dried                    0.0   0.0   0.0   0.0   0.0   0.0   0.0   

                                         
CustomerID                        98200  
ProductName                              
Anchovy Paste - 56 G Tube           0.0  
Appetizer - Mini Egg Roll, Shrimp   0.0  
Appetizer - Mushroom Tart           0.0  
Appetizer - Sausage Rolls           0.0  
Apricots - Dried                    0.0  

[5 rows x 1000 columns]

In [294]:
result = pd.concat([cust_mat['Quantity'], ranked], axis=1, sort=False)
result = result[result[33]==0]['Quantity'].sort_values(ascending=False)
result

Wine - Blue Nun Qualitatswein        3.0
Butter - Unsalted                    3.0
Soup - Campbells Bean Medley         3.0
Wine - Ej Gallo Sierra Valley        3.0
Yoghurt Tubes                        2.0
Beets - Candy Cane, Organic          2.0
Nut - Chestnuts, Whole               2.0
Steam Pan - Half Size Deep           2.0
Pecan Raisin - Tarts                 2.0
Cheese - Boursin, Garlic / Herbs     2.0
Sea Bass - Whole                     2.0
Scallops 60/80 Iqf                   2.0
Cheese Cloth No 100                  2.0
Chicken - Soup Base                  2.0
Sausage - Breakfast                  2.0
Tea - Decaf Lipton                   2.0
Juice - Lime                         2.0
Chocolate - Dark                     2.0
Cod - Black Whole Fillet             2.0
Sardines                             2.0
Coffee - Dark Roast                  2.0
Coffee Decaf Colombian               2.0
Raspberries - Fresh                  2.0
Cup - Translucent 7 Oz Clear         2.0
Crab - Dungeness

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.